In [ ]:
sc.install_pypi_package("pybind11==2.10.3")
sc.install_pypi_package("numpy==1.19.0")
sc.install_pypi_package("Pillow==8.2")
sc.install_pypi_package("Cython==0.29.33")
sc.install_pypi_package("scipy==1.2.0")
sc.install_pypi_package("pythran==0.12.1")
sc.install_pypi_package("pandas==1.0.0")
sc.install_pypi_package("matplotlib==3.3.0")
sc.install_pypi_package("lifelines==0.27.4")
sc.install_pypi_package("s3fs==0.4.2")
sc.install_pypi_package("boto3==1.26.59")
sc.install_pypi_package("joblib==1.2.0")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from itertools import groupby
from operator import itemgetter

from lifelines import CoxPHFitter

import pandas as pd
import boto3
import os
import pickle
import tempfile
import joblib
import io
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

MongoUrl = "mongodb+srv://xxxxxxxxxxxxxxxxxxxxxx/"

In [ ]:
spark = SparkSession \
    .builder \
    .appName("FinancialSheets_ML_Training") \
    .config("spark.cores.max", 6) \
    .config("spark.executor.cores", 6) \
    .config("spark.executor.memory", "36g") \
    .getOrCreate()

In [ ]:
df = spark.read.format("mongodb") \
    .option("spark.mongodb.read.connection.uri", MongoUrl) \
    .option("spark.mongodb.write.connection.uri", MongoUrl) \
    .option("database", "coreEngine") \
    .option("collection", "ReportFeatures_temp") \
    .load()

In [ ]:
features = df.toPandas()

In [ ]:
hf_features = features.drop(["_id"], axis=1)
hf_features["event"] = hf_features["event"].fillna(0).astype(bool)
hf_features = hf_features.sort_values(["stockCode", "rceptNo"])
hf_features = hf_features[hf_features["period"] >= 1.0]

In [ ]:
columns = [
    "CACL", "CATA", "CLCA", "CLGR", "CLTL", "DSR01", "DSR02", "DSR03", 
    "DSR04", "DSR05", "DSR06", "DSR07", "DSR08", "EBTIN", "EQTA", "FAGR", 
    "FFOEQ", "FFOTL", "INSL", "INTL", "LNSL", "LNTA", "MB", "NEGBE", 
    "NIGR", "NISL", "RETA", "SLEQ", "SLFA", "TLEQ", "TLTA"]

for column in columns:
    hf_features[column]=hf_features.groupby(["bsnsYear", "reprtCode"])[column].apply(lambda x:x.fillna(0))

In [ ]:
hf_features = hf_features.dropna()

In [ ]:
boundry = "2020"

df_train_set = hf_features[hf_features["bsnsYear"] <= boundry].drop_duplicates()
df_train_set = df_train_set.set_index(["stockCode", "corpCls", "corpCode", "reprtCode", "rceptNo", "stockName", "updateDate", "bsnsYear", "quarter"])

In [ ]:
model = CoxPHFitter(penalizer=0.0001, l1_ratio=1)
model.fit(df_train_set, duration_col="period", event_col="event", fit_options=dict(step_size=0.1))

In [ ]:
model.print_summary()

In [ ]:
s3 = boto3.client(
    "s3",
    region_name="ap-northeast-2",
    aws_access_key_id="xxxxxxxxxxxxxxxxxxxxxx",
    aws_secret_access_key="xxxxxxxxxxxxxxxxxxxxxx")

In [ ]:
"""
f = io.BytesIO()
joblib.dump(model, f)
f.seek(0)
s3.put_object(Bucket="penta-engine", Key="FinancialSheetsModel.pkl", Body=f)
"""

In [ ]:
spark.stop()